In [1]:
import pandas as pd
import json
import os

In [2]:
years = [2018, 2019, 2020, 2021, 2022, 2023]
years

[2018, 2019, 2020, 2021, 2022, 2023]

In [3]:
for year in years:
    file_rename_path = f'../Project/{year}'
    
    for file in os.listdir(file_rename_path):
        full_path = os.path.join(file_rename_path, file)
        if not file.endswith(".json") and os.path.isfile(full_path):
            new_name = file + ".json"
            new_full_path = os.path.join(file_rename_path, new_name)
            os.rename(full_path, new_full_path)
            print(f"Renamed the file {file} to {new_name}")

#### Test reading file data 


In [4]:
df = pd.read_json('../Project/2018/201800000.json')

In [5]:
df.shape

(8, 1)

In [6]:
df.describe()

,abstracts-retrieval-response
count,6
unique,6
top,{'ait:process-info': {'ait:status': {'@state':...
freq,1


In [7]:
df.columns

Index(['abstracts-retrieval-response'], dtype='object')

In [8]:
print(df.head())

                                  abstracts-retrieval-response
item         {'ait:process-info': {'ait:status': {'@state':...
affiliation  [{'affiliation-city': 'Stanford', '@id': '6003...
coredata     {'srctype': 'b', 'eid': '2-s2.0-85077976956', ...
idxterms                                                  None
language                                  {'@xml:lang': 'eng'}


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, item to authors
Data columns (total 1 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   abstracts-retrieval-response  6 non-null      object
dtypes: object(1)
memory usage: 128.0+ bytes


In [10]:
df

,abstracts-retrieval-response
item,{'ait:process-info': {'ait:status': {'@state':...
affiliation,"[{'affiliation-city': 'Stanford', '@id': '6003..."
coredata,"{'srctype': 'b', 'eid': '2-s2.0-85077976956', ..."
idxterms,None
language,{'@xml:lang': 'eng'}
authkeywords,None
subject-areas,"{'subject-area': [{'@_fa': 'true', '$': 'Medic..."
authors,"{'author': [{'ce:given-name': 'Krit', 'preferr..."


In [11]:
for i in df['abstracts-retrieval-response'].get('authors').get('author'):
    print(i.get('ce:indexed-name'))

Pongpirul K.
Lungren M.P.


In [12]:
for i in df['abstracts-retrieval-response'].items():
    print(i)

('item', {'ait:process-info': {'ait:status': {'@state': 'update', '@type': 'core', '@stage': 'S300'}, 'ait:date-delivered': {'@day': '19', '@timestamp': '2020-05-19T23:06:15.000015-04:00', '@year': '2020', '@month': '05'}, 'ait:date-sort': {'@day': '31', '@year': '2018', '@month': '12'}}, 'bibrecord': {'head': {'author-group': [{'affiliation': {'country': 'Thailand', '@afid': '60028190', '@country': 'tha', 'city': 'Bangkok', 'organization': [{'$': 'Department of Preventive and Social Medicine'}, {'$': 'Faculty of Medicine'}, {'$': 'Chulalongkorn University'}], 'affiliation-id': {'@afid': '60028190', '@dptid': '104425678'}, '@dptid': '104425678'}, 'author': [{'ce:given-name': 'Krit', 'preferred-name': {'ce:given-name': 'Krit', 'ce:initials': 'K.', 'ce:surname': 'Pongpirul', 'ce:indexed-name': 'Pongpirul K.'}, '@seq': '1', 'ce:initials': 'K.', '@_fa': 'true', '@type': 'auth', 'ce:degrees': 'PhD..', 'ce:surname': 'Pongpirul', '@auid': '14720203700', 'ce:indexed-name': 'Pongpirul K.'}]}, {

In [13]:
lang = df['abstracts-retrieval-response'].language.get('@xml:lang')
lang

'eng'

In [14]:
# df['abstracts-retrieval-response'].coredata

In [15]:
data_rows = []

In [16]:
for year in years:
    folder_path = f"../Project/{year}"
    for file_name in os.listdir(folder_path):
        if file_name.startswith(str(year)) and file_name.endswith(".json"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r", encoding="utf-8") as file:
                try:
                    data = json.load(file)

                    author_groups = (
                        data.get("abstracts-retrieval-response", {})
                        .get("item", {})
                        .get("bibrecord", {})
                        .get("head", {})
                        .get("author-group", [])
                    )

                    countries = []
                    for author in author_groups:
                        affiliation = author.get("affiliation", {})
                        country = affiliation.get("country", "Unknown")
                        countries.append(country)

                    countries_string = ",".join(countries)

                    auth_keywords = (
                        data.get("abstracts-retrieval-response", {})
                        .get("authkeywords", {})
                        .get("author-keyword", [])
                    )

                    keywords = [
                        keyword.get("$", "")
                        for keyword in auth_keywords
                        if isinstance(keyword, dict)
                    ]
                    keywords_string = ",".join(keywords) if keywords else "null"

                    row_data = pd.json_normalize(
                        data.get("abstracts-retrieval-response", {})
                    )
                    row = {
                        col: row_data[col].iloc[0] if col in row_data else pd.NA
                        for col in row_data.columns
                    }

                    row["item.bibrecord.head.author-group.affiliation.country"] = (
                        countries_string
                    )
                    row["authkeywords.author-keyword"] = keywords_string

                    data_rows.append(row)

                except Exception as e:
                    continue
                    # print(f"Error processing file {file_name}: {e}")

In [24]:
# data_rows

In [25]:
df = pd.DataFrame(data_rows)

In [26]:
df.to_csv('../MergedData.csv', index=False)

In [27]:
df

,affiliation,item.ait:process-info.ait:status.@state,item.ait:process-info.ait:status.@type,item.ait:process-info.ait:status.@stage,item.ait:process-info.ait:date-delivered.@day,item.ait:process-info.ait:date-delivered.@year,item.ait:process-info.ait:date-delivered.@timestamp,item.ait:process-info.ait:date-delivered.@month,item.ait:process-info.ait:date-sort.@day,item.ait:process-info.ait:date-sort.@year,...,item.bibrecord.tail.bibliography.reference.ref-info.ref-text,item.bibrecord.head.source.contributor-group.contributor.ce:e-address.$,item.bibrecord.head.source.contributor-group.contributor.ce:e-address.@type,item.bibrecord.head.enhancement.descriptorgroup.descriptors.@controlled,item.bibrecord.head.enhancement.descriptorgroup.descriptors.@type,item.bibrecord.head.enhancement.descriptorgroup.descriptors.descriptor,item.bibrecord.head.source.volisspag.pages,item.bibrecord.head.related-item.source.part,item.bibrecord.tail.bibliography.reference.@reference-instance-id,item.bibrecord.tail.bibliography.reference.ref-info.refd-itemidlist.itemid
0,"[{'affiliation-city': 'Bangkok', '@id': '60091...",update,core,S300,13,2023,2023-07-13T01:13:22.000022-04:00,07,01,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,update,core,S300,18,2020,2020-11-18T00:19:29.000029-05:00,11,01,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'affiliation-city': 'Khon Kaen', '@id': '600...",update,core,S300,29,2022,2022-07-29T19:42:21.000021-04:00,07,01,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'affiliation-city': 'Bangkok', '@id': '60110...",new,core,S300,17,2020,2020-12-17T18:02:55.000055-05:00,12,01,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[{'affiliation-city': 'Bangkok', '@id': '60138...",update,core,S300,20,2022,2022-04-20T02:40:38.000038-04:00,04,01,2018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14155,"[{'affiliation-city': 'Fort Collins', '@id': '...",update,core,S300,13,2023,2023-07-13T01:47:42.000042-04:00,07,01,2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14156,"[{'affiliation-city': 'Bangkok', '@id': '60028...",update,core,S300,28,2023,2023-04-28T12:51:12.000012-04:00,04,01,2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14157,"[{'affiliation-city': 'Bangkok', '@id': '60199...",update,core,S300,02,2023,2023-04-02T18:09:17.000017-04:00,04,01,2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14158,NaN,new,core,S300,28,2023,2023-07-28T09:25:21.000021-04:00,07,01,2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
